In [1]:
import os, re
v_transformers = !pip show transformers
v_transformers = "\n".join(v_transformers)
v_transformers = re.search(r"version\:\s*([\d\.]+)", v_transformers.lower()).group(1).strip()
print("initial transformers version:", v_transformers)
if v_transformers != "4.20.1":
#     os.system('pip show transformers')
    os.system('pip uninstall -y transformers')
    os.system('pip uninstall -y tokenizers')
    os.system('python -m pip install --no-index --find-links=../input/pip-wheels-transformers-4-20-1/ transformers')
    os.system('python -m pip install --no-index --find-links=../input/pip-wheels-transformers-4-20-1/ tokenizers')
    import tokenizers
    import transformers
    print(f"tokenizers.__version__: {tokenizers.__version__}")
    print(f"transformers.__version__: {transformers.__version__}")
else:
    print("transformers is already up to date")

initial transformers version: 4.20.1
transformers is already up to date


In [2]:
import os, sys
sys.path.insert(0, "../input/kkillerfpzeff2ndplace/src")

In [3]:
import pandas as pd, numpy as np
import torch

from tqdm.auto import tqdm
from pathlib import Path
from  datetime import datetime


import configs
import dataset
import inference

import gc

In [4]:
IS_DEBUG = False
IS_RERUN = ( len(pd.read_csv("../input/feedback-prize-effectiveness/test.csv", usecols=["essay_id"])) > 100 )

os.environ["TOKENIZERS_PARALLELISM"] = "false"

if IS_DEBUG:
    configs.TRAIN_ROOT = "../input/feedback-prize-effectiveness/train"
    configs.TRAIN_CSV_PATH = "../input/feedback-prize-effectiveness/train.csv"
    configs.SAMPLE_SUB_CSV_PATH = None
else:
    configs.TRAIN_ROOT = "../input/feedback-prize-effectiveness/test"
    configs.TRAIN_CSV_PATH = "../input/feedback-prize-effectiveness/test.csv"
    configs.SAMPLE_SUB_CSV_PATH = "../input/feedback-prize-effectiveness/sample_submission.csv"


DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(DEVICE)

cuda


# WEIGHTS

In [5]:
params = [
   
    fprize_efgprint.src.inference.get_params(
        
        model_name="microsoft/deberta-v2-xlarge",
        use_mixup=False,
        forward_type="forward_1span",
        use_token_types=True,
        use_layer_norm=True,
        batch_size=4,
        maxlen=1792,
        num_workers=0,
        weight=.15,
        
        config_path="../input/kkiller-transformers-data/microsoft_deberta-v2-xlarge-mnli",
        tokenizer_path="../input/kkiller-transformers-data/microsoft_deberta-v2-xlarge-mnli",
        
        is_pickle=False,
        device=DEVICE,
        
        model_paths=[
            "../input/fpzeff-db2xl-psl-1span-notune-v3-1-m1152-weights/db2xl_psl_1span_v3_1_m1152/fpeff_microsoft_deberta-v2-xlarge_fold0_epoch_02_loss_v2_val_0.5704_20220820152247.pth",
            "../input/fpzeff-db2xl-psl-1span-notune-v3-1-m1152-weights/db2xl_psl_1span_v3_1_m1152/fpeff_microsoft_deberta-v2-xlarge_fold1_epoch_02_loss_v2_val_0.6005_20220820194712.pth",
        ],
        
        oof_name="oof_db2xl_fgp_psl_v3_1_1152_220823",
        
        dataset_module=fprize_efgprint.src.dataset,
        inference_module=fprize_efgprint.src.inference,
    ),
    
    
    fprize_efgprint.src.inference.get_params(
        model_name="microsoft/deberta-xlarge",
        use_mixup=False,
        forward_type="forward_1span",
        use_token_types=True,
        use_layer_norm=True,
        batch_size=4,
        maxlen=1792,
        num_workers=0,
        weight=.25,
        config_path="../input/kkiller-transformers-data/microsoft_deberta-xlarge",
        tokenizer_path="../input/kkiller-transformers-data/microsoft_deberta-xlarge",
        is_pickle=False,
        device=DEVICE,
        model_paths=list(Path("../input/fpzeff-db1xl-psl-1span-notune-v3-1-m864-weights/db1xl_psl_1span_v3_1_m864/").glob("*.pth")),
        
        oof_name="oof_db1xl_fgp_psl_v3_1_864_220823",
        
        dataset_module=fprize_efgprint.src.dataset,
        inference_module=fprize_efgprint.src.inference,
    ),
    
    
    fprize_efgprint.src.inference.get_params(
        model_name="microsoft/deberta-large",
        use_mixup=False,
        forward_type="forward_1span",
        use_token_types=True,
        use_layer_norm=True,
        batch_size=8,
        maxlen=1792,
        num_workers=0,
        weight=.25,
        config_path="../input/kkiller-transformers-data/microsoft_deberta-large",
        tokenizer_path="../input/kkiller-transformers-data/microsoft_deberta-large",
        is_pickle=False,
        device=DEVICE,
        model_paths=list(Path("../input/fpzeff-db1l-psl-1span-notune-v3-1-m1152-weights/db1l_psl_1span_v3_1_m1152/").glob("*.pth")),
        
        oof_name="oof_db1l_fgp_psl_v3_1_1152_220823",
        
        dataset_module=fprize_efgprint.src.dataset,
        inference_module=fprize_efgprint.src.inference,
    ),
    
    
    fprize_efgprint.src.inference.get_params(
        model_name="microsoft/deberta-v3-large",
        use_mixup=False,
        forward_type="forward_1span",
        use_token_types=True,
        use_layer_norm=True,
        batch_size=8,
        maxlen=1792,
        num_workers=0,
        weight=.35,
        config_path="../input/kkiller-transformers-data/microsoft_deberta-v3-large",
        tokenizer_path="../input/kkiller-transformers-data/microsoft_deberta-v3-large",
        is_pickle=False,
        device=DEVICE,
        model_paths=(
            list(Path("../input/fpzeff-db3l-psl-1span-notune-v3-1-m1152-weights/db3l_psl_1span_v3_1_m1152/").glob("*.pth"))
            + ["../input/fpzeff-db3l-psl-nofold-1span-v3-1-m1152-weights/db3l_psl_no_fold_1span_v3_1_m1152/fpeff_microsoft_deberta-v3-large_fold0_epoch_02_loss_v2_val_0.3842_20220822215539.pth"]
        ),
        
        oof_name="oof_full_db3l_fgp_psl_v3_1_1152_220822",
        
        dataset_module=fprize_efgprint.src.dataset,
        inference_module=fprize_efgprint.src.inference,
    ),
    
    
]

# for param in params:
#     param["weight"] =  WEIGHTS[REDUCED_MODEL_NAMES[param["model_paths"][0].parent.stem]] if param["model_paths"][0] is not None else 1.0

S = sum([param["weight"] for param in params])
assert abs(S- 1.0) < 1e-3, S
# params[0]

print("TOTAL UNIQUE MODELS:", len(params))
print("TOTAL  CHECKPOINTS:", sum([len(param["model_paths"]) for param in params]))
print("MODELS PER CHKPT;", [len(param["model_paths"]) for param in params])

if not IS_RERUN:
    for param in params:
        param["model_paths"] = param["model_paths"][:1]

/opt/conda/lib/python3.7/site-packages/transformers/convert_slow_tokenizer.py:435: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  "The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option"


TOTAL UNIQUE MODELS: 4
TOTAL  CHECKPOINTS: 17
MODELS PER CHKPT; [2, 4, 5, 6]


In [6]:
if IS_DEBUG:
    df = dataset.read_train_csv(configs.TRAIN_CSV_PATH, nrows=None, is_test=True)#.sample(10_000).reset_index(drop=True)
else:
    df = dataset.read_train_csv(configs.TRAIN_CSV_PATH, is_test=True)

essays = {uuid: dataset.read_from_id(uuid) for uuid in df["id"].unique()}

df["nchars"] = [len(essays[uuid]) for uuid in df["id"]]
df["is_true_obs"] = True

uuids = dict(zip(df["id"], -df["nchars"]))
uuids = sorted(list(uuids.keys()), key=lambda uuid: uuids[uuid])

if IS_DEBUG:
    uuids = uuids[:200]

print("nuuids", len(uuids))

print(f"len(uuids): {len(uuids)}")

print(df.shape)
df.head()

nuuids 1
len(uuids): 1
(10, 7)


,discourse_id,essay_id,discourse_text,discourse_type,id,nchars,is_true_obs
0,a261b6e14276,D72CB1C11673,Making choices in life can be very difficult. ...,Lead,D72CB1C11673,2439,True
1,5a88900e7dc1,D72CB1C11673,Seeking multiple opinions can help a person ma...,Position,D72CB1C11673,2439,True
2,9790d835736b,D72CB1C11673,it can decrease stress levels,Claim,D72CB1C11673,2439,True
3,75ce6d68b67b,D72CB1C11673,a great chance to learn something new,Claim,D72CB1C11673,2439,True
4,93578d946723,D72CB1C11673,can be very helpful and beneficial.,Claim,D72CB1C11673,2439,True


In [7]:
sub = None
cols = ["Ineffective", "Adequate", "Effective"]
if not IS_DEBUG:
    sample_sub = pd.read_csv(configs.SAMPLE_SUB_CSV_PATH)
    
for param in params:
    if param["model_paths"][0] is not None:
        print("{}: {}".format(Path(param["model_paths"][0]).parent.stem,  len(param["model_paths"])))
        
    inference_module = param.get("inference_module", inference)
    
    temp  = inference_module.predict_from_param(
        uuids=uuids,
        param=param,
        df=df,
        essays=essays,
#         agg_func=AGG_FUNC,
        bar_for_models=IS_DEBUG,
#         bar_for_data=IS_DEBUG,
        bar_for_data=False,
    )
    
    if not IS_DEBUG:
        temp = sample_sub[["discourse_id"]].merge(temp, on="discourse_id", how="left")
        temp[cols] = temp[cols].fillna(1/3.)
    
    temp[cols] *= param["weight"]
    temp.sort_values(["id", "discourse_id"], inplace=True)
    temp.reset_index(drop=True, inplace=True)
    
    if sub is None:
        sub = temp
    else:
        assert len(sub) == len(temp)
        sub[cols] += temp[cols]
    

if not IS_DEBUG:
    sub = sample_sub[["discourse_id"]].merge(sub, on="discourse_id", how="left")
    sub[cols] = sub[cols].fillna(1/3.)

sub[cols] = np.clip(sub[cols].values, 1e-5, 1-1e-5)
sub[["discourse_id", *cols]].to_csv("submission.csv", index=False)

print(sub.shape)

sub.head(30)

db2xl_psl_1span_v3_1_m1152: 1


  0%|          | 0/1 [00:00<?, ?it/s]

db1xl_psl_1span_v3_1_m864: 1


  0%|          | 0/1 [00:00<?, ?it/s]

db1l_psl_1span_v3_1_m1152: 1


  0%|          | 0/1 [00:00<?, ?it/s]

db3l_psl_1span_v3_1_m1152: 1


  0%|          | 0/1 [00:00<?, ?it/s]

(10, 5)


,discourse_id,id,Ineffective,Adequate,Effective
0,a261b6e14276,D72CB1C11673,0.006379,0.243497,0.750124
1,5a88900e7dc1,D72CB1C11673,0.034001,0.651887,0.314112
2,9790d835736b,D72CB1C11673,0.045995,0.430916,0.523089
3,75ce6d68b67b,D72CB1C11673,0.053684,0.427175,0.519141
4,93578d946723,D72CB1C11673,0.143364,0.436776,0.419860
5,2e214524dbe3,D72CB1C11673,0.017002,0.386508,0.596489
6,84812fc2ab9f,D72CB1C11673,0.017972,0.345081,0.636948
7,c668ff840720,D72CB1C11673,0.069477,0.504962,0.425561
8,739a6d00f44a,D72CB1C11673,0.038264,0.438633,0.523103
9,bcfae2c9a244,D72CB1C11673,0.007276,0.513454,0.479269
